In [ ]:
from datetime import datetime

import wandb
from sklearn import datasets
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_validate, RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier

from assignment1.bagging_id3 import MyBaggingID3


In [ ]:
! wandb login

In [ ]:
def evaluate_model(model, ds_name, X, y, x_test, y_test):
    # Define the evaluation metrics
    scoring = {
        'accuracy': 'accuracy',
        'precision': 'precision',
        'recall': 'recall',
        'f1_score': 'f1',
        'roc_auc_score': 'roc_auc'
    }
    # Define the cross-validation procedure
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=42)
    # Evaluate the models
    cv_results = cross_validate(model, X, y, cv=cv, scoring=scoring, return_train_score=False)
    model_config = model.get_params()
    wandb.init(project="Assigment1", name=f"{model.__class__.__name__}_{ds_name}_{datetime.now()}",
               config=model_config)
    wandb.config.update({'fit_time': cv_results['fit_time']})
    wandb.log({metric: cv_results['test_%s' % metric].mean() for metric in scoring})
    # y_pred = model.predict(x_test)

    wandb.finish()

In [ ]:
n_estimetors = 250
max_samples = 1
max_features = 0.4
max_depth = 50

In [ ]:
# Define the models
my_bagging_id3 = MyBaggingID3(n_estimators=n_estimetors, max_samples=max_samples,
                              max_features=max_features, max_depth=max_depth)
dtc = DecisionTreeClassifier()
bc = BaggingClassifier(base_estimator=dtc, n_estimators=n_estimetors)

In [ ]:
# Dataset 1 -  The 20 newsgroups text dataset
X, Y = datasets.fetch_20newsgroups_vectorized(return_X_y=True)
x_test, y_test = datasets.fetch_20newsgroups_vectorized(subset='test', return_X_y=True)